# Visual Analytics

## Assignment 3

**Instructor:** Dr. Marco D'Ambros  
**TAs:** Carmen Armenti, Mattia Giannaccari

**Contacts:** marco.dambros@usi.ch, carmen.armenti@usi.ch, mattia.giannaccari@usi.ch

**Due Date:** May 16, 2025 @ 23:55

---
The goal of this assignment is to use **Spark (PySpark)** and **Polars** in Jupyter notebooks.  
The files `trip_data.csv`, `trip_fare.csv`, and `nyc_boroughs.geojson` are available in the provided folder: [Assignment3-data](https://usi365-my.sharepoint.com/:f:/g/personal/armenc_usi_ch/Ejp7sb8QAMROoWe0XUDcAkMBoqUFk-w2Vgroup025NhAww?e=2I7SMC).

You may clean the data as needed; however, please note that specific data cleaning steps will be required in **Exercise 5**. If you choose to clean the data before Exercise 5, make sure to retain the **original dataset** for use with the Polars exercises.

- Use **Spark** to solve **Exercises 1–4**
- Use **Polars** to solve **Exercises 5–8**

You are encouraged to use [Spark window functions](https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-window.html) whenever appropriate.

Please name your notebook file as `SurnameName_Assignment3.ipynb`

# Imports

In [51]:
import json
from pathlib import Path
from typing import Any

import geopandas as gpd
import polars as pl
from bokeh.io import output_notebook
from bokeh.layouts import row
from bokeh.models import (BasicTicker, ColorBar, ColumnDataSource, HoverTool,
                          LinearColorMapper, PrintfTickFormatter)
from bokeh.palettes import Reds
from bokeh.plotting import figure, output_notebook, show
from bokeh.transform import linear_cmap, transform
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, lag
from pyspark.sql.functions import max as max_fun
from pyspark.sql.functions import min as min_fun
from pyspark.sql.functions import when
from pyspark.sql.types import DateType
from pyspark.sql.window import Window

output_notebook()

Loading BokehJS ...

## Spark

### Exercise 1
Join the `trip_data` and `trip_fare` dataframes into one and consider only data on 2013-01-01. Please specify the number of rows obtained after joining the 2 datasets.

In [52]:
# setup spark session
session = SparkSession.builder.getOrCreate()
session.conf.set("spark.sql.shuffle.partitions", 400)
session.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [53]:
# load data for trips and fares
trip_data_df = session.read.option("inferSchema", "true").option("header", "true").csv("./datasets/trip_data.csv")
trip_fare_df = session.read.option("inferSchema", "true").option("header", "true").csv("./datasets/trip_fare.csv")

In [54]:
# normalize column names by stripping whitespace
trip_data_df.select([col(c).alias(c.strip()) for c in trip_data_df.columns]) 
trip_fare_df = trip_fare_df.select([col(c).alias(c.strip()) for c in trip_fare_df.columns])

In [55]:
# extract the pickup and dropoff date from the datetime columns
trip_data_df = trip_data_df.select(
    "*",
    col("pickup_datetime").cast(DateType()).alias("pickup_date"),
    col("dropoff_datetime").cast(DateType()).alias("dropoff_date")
)

In [56]:
# Join trip data with fare data on medallion, hack_license, and pickup_datetime
joined_df = trip_data_df.join(trip_fare_df, on=["medallion", "hack_license", "pickup_datetime"])

In [57]:
# filter the joined DataFrame for 2013-01-01 (I prefer to do it now so I filter once instead of twice)
filtered_df = joined_df.filter(
    (joined_df.pickup_date == "2013-01-01") &
    (joined_df.dropoff_date == "2013-01-01")
).drop("vendor_id", "store_and_fwd_flag")

In [58]:
print("Number of records for 2013-01-01: ", filtered_df.count())

Number of records for 2013-01-01:  410816


In [59]:
#print schema
filtered_df.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- rate_code: integer (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- pickup_date: date (nullable = true)
 |-- dropoff_date: date (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)



### Exercise 2
Provide a graphical representation to compare the average fare amount for trips _within_ and _across_ all the boroughs. You may want to have a look at: https://docs.bokeh.org/en/latest/docs/user_guide/topics/categorical.html#categorical-heatmaps

In [60]:
# load data
nyc_df = gpd.read_file('./datasets/nyc-boroughs.geojson')

In [61]:
# convert to pandas because geopandas does not support spark dataframes
filtered_df_pandas = filtered_df.toPandas()

# create 2 GeoDataFrames for pickup and dropoff locations
pickup_gdf = gpd.GeoDataFrame(
    filtered_df_pandas,
    geometry=gpd.points_from_xy(filtered_df_pandas['pickup_longitude'], filtered_df_pandas['pickup_latitude']),
    crs=nyc_df.crs
)

dropoff_gdf = gpd.GeoDataFrame(
    filtered_df_pandas,
    geometry=gpd.points_from_xy(filtered_df_pandas['dropoff_longitude'], filtered_df_pandas['dropoff_latitude']),
    crs=nyc_df.crs
)

In [62]:
# spatial join to find the boroughs for pickup and dropoff locations
pickup_boroughs = gpd.sjoin(pickup_gdf, nyc_df, how="left", predicate="within")
dropoff_boroughs = gpd.sjoin(dropoff_gdf, nyc_df, how="left", predicate="within")

In [63]:
filtered_df_pandas["pickup_borough"] = pickup_boroughs["borough"].values
filtered_df_pandas["dropoff_borough"] = dropoff_boroughs["borough"].values

In [64]:
filtered_df_pandas.head()

,medallion,hack_license,pickup_datetime,rate_code,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,...,dropoff_date,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,pickup_borough,dropoff_borough
0,00005007A9F30E289E760362F69E4EAD,43468C5D35F828693D96CB7CC9FDF341,2013-01-01 06:48:43,1,2013-01-01 06:50:38,1,114,0.4,-73.975136,40.752312,...,2013-01-01,DIS,3.5,0.0,0.5,0.0,0.0,4.0,Manhattan,Manhattan
1,00005007A9F30E289E760362F69E4EAD,43468C5D35F828693D96CB7CC9FDF341,2013-01-01 10:04:50,1,2013-01-01 10:12:33,1,463,1.5,-74.008255,40.736687,...,2013-01-01,CSH,8.0,0.0,0.5,0.0,0.0,8.5,Manhattan,Manhattan
2,00005007A9F30E289E760362F69E4EAD,43468C5D35F828693D96CB7CC9FDF341,2013-01-01 12:31:47,1,2013-01-01 12:39:41,1,473,2.3,-73.967064,40.760773,...,2013-01-01,CSH,9.0,0.0,0.5,0.0,0.0,9.5,Manhattan,Manhattan
3,00005007A9F30E289E760362F69E4EAD,43468C5D35F828693D96CB7CC9FDF341,2013-01-01 15:13:35,1,2013-01-01 15:31:07,1,1052,3.4,-73.997833,40.716984,...,2013-01-01,CSH,14.5,0.0,0.5,0.0,0.0,15.0,Manhattan,Manhattan
4,00005007A9F30E289E760362F69E4EAD,43468C5D35F828693D96CB7CC9FDF341,2013-01-01 16:14:19,1,2013-01-01 16:18:46,1,266,0.8,-73.978615,40.771908,...,2013-01-01,CSH,5.0,0.0,0.5,0.0,0.0,5.5,Manhattan,Manhattan


In [65]:
# create spark dataframe from pandas dataframe
spark_df = session.createDataFrame(filtered_df_pandas)

# dataframe to find the average fare amount for each trip group
trip_group_df = spark_df \
    .groupBy(['pickup_borough', 'dropoff_borough']) \
    .avg('fare_amount') \
    .withColumnRenamed('avg(fare_amount)', 'avg_fare')

unique_borough = nyc_df['borough'].unique()

In [66]:
# extract the min and max fare amount for the color mapper
min_val, max_val = trip_group_df.select(min_fun('avg_fare'), max_fun('avg_fare')).first()

p = figure(
    title="Average Fare Amount for Pickup and Dropoff Boroughs",
    x_range=unique_borough,
    y_range=unique_borough,
    width=1000,
    height=500,
    tools="hover",
    toolbar_location='below',
    tooltips=[('Average Fare Amount', '@avg_fare{0.2f}')]
)

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0

mapper = linear_cmap(field_name="avg_fare", palette=Reds[7][::-1], low=min_val, high=max_val)

source_df = trip_group_df.toPandas()
p.rect(
    x="pickup_borough", y="dropoff_borough",
    width=1, height=1,
    source=source_df,
    fill_color=mapper,
    line_color=None
)

color_bar = ColorBar(
    color_mapper=mapper['transform'],
    formatter=PrintfTickFormatter(format="%d%%"),
    border_line_color=None,
    padding=8
)
p.add_layout(color_bar, 'right')

show(p)

25/05/15 16:50:27 WARN TaskSetManager: Stage 62 contains a task of very large size (8538 KiB). The maximum recommended task size is 1000 KiB.
25/05/15 16:50:28 WARN TaskSetManager: Stage 68 contains a task of very large size (8538 KiB). The maximum recommended task size is 1000 KiB.


### Exercise 3
Consider only Manhattan, Bronx and Brooklyn boroughs. Then create a dataframe that shows the total number of trips *within* the same borough and *across* all the other boroughs mentioned before (Manhattan, Bronx, and Brooklyn) where the passengers are more or equal than 3.

For example, for Manhattan borough you should consider the total number of the following trips:
- Manhattan → Manhattan
- Manhattan → Bronx
- Manhattan → Brooklyn

You should then do the same for Bronx and Brooklyn boroughs.

In [67]:
# filter the dataframe for specific boroughs and passenger count
boroughs = ["Manhattan", "Bronx", "Brooklyn"]

filtered_df = spark_df.filter(
    (col("pickup_borough").isin(boroughs)) &
    (col("dropoff_borough").isin(boroughs)) &
    (col("passenger_count") >= 3)
)

In [68]:
# add a new column to the dataframe to indicate the trip type
labeled_df = filtered_df.withColumn(
    "trip_type",
    when(col("pickup_borough") == col("dropoff_borough"), "within")
    .otherwise("across")
)

In [69]:
# count the number of trips for each trip type and borough combination
grouped_borough_df = labeled_df.groupBy(
    ["pickup_borough", "dropoff_borough", "trip_type"]
).count()

grouped_borough_df.show(5)

25/05/15 16:50:28 WARN TaskSetManager: Stage 71 contains a task of very large size (8538 KiB). The maximum recommended task size is 1000 KiB.


+--------------+---------------+---------+-----+
|pickup_borough|dropoff_borough|trip_type|count|
+--------------+---------------+---------+-----+
|         Bronx|      Manhattan|   across|   55|
|      Brooklyn|          Bronx|   across|   10|
|     Manhattan|       Brooklyn|   across| 2762|
|     Manhattan|      Manhattan|   within|62391|
|      Brooklyn|       Brooklyn|   within| 2014|
+--------------+---------------+---------+-----+
only showing top 5 rows



### Exercise 4
Create a dataframe where each row represents a driver, and there is one column per borough.
For each driver-borough, the dataframe provides the maximum number of consecutive trips
for the given driver, within the given borough. Please consider only trips which were payed by card. 

For example, if for driver A we have (sorted by time):
- Trip 1: Bronx → Bronx
- Trip 2: Bronx \→ Bronx
- Trip 3: Bronx → Manhattan
- Trip 4: Manhattan → Bronx.
    
The maximum number of consecutive trips for Bronx is 2.

In [70]:
# filter dataframe for trips with credit card payment type and same pickup and dropoff borough
trip_drivers_df = spark_df.filter((col("payment_type") == "CRD") 
            & (col("pickup_borough") == col("dropoff_borough"))
            & col("pickup_borough").isNotNull()
            & (col("pickup_borough") != "")
            & ~isnan(col("pickup_borough")))

unique_boroughs_df = trip_drivers_df.select("pickup_borough").distinct()
unique_boroughs = [row["pickup_borough"] for row in unique_boroughs_df.collect()]
print(unique_boroughs)

25/05/15 16:50:28 WARN TaskSetManager: Stage 74 contains a task of very large size (8538 KiB). The maximum recommended task size is 1000 KiB.


['Manhattan', 'Queens', 'Brooklyn', 'Bronx', 'Staten Island']


In [71]:
# Define the window
w = Window.partitionBy('hack_license').orderBy('pickup_datetime')

# Create flag and borough columns
for borough in unique_boroughs:
    flag_col = f"{borough}_flag"

    # Create a flag column for each borough and set it to 1 if the trip is within that borough else 0
    trip_drivers_df = trip_drivers_df.withColumn(
        flag_col,
        when(
            (col("pickup_borough") == borough) &
            (col("dropoff_borough") == borough) &
            (col("payment_type") == "CRD"),
            1
        ).otherwise(0)
    )
    
    # Create a new column for each borough and set it to the lag of the flag colum
    trip_drivers_df = trip_drivers_df.withColumn(
        borough,
        when(
            col(flag_col) == 1,
            lag(col(flag_col) + 1, 1, 0).over(w)
        ).otherwise(0)
    )

grouped_df = trip_drivers_df.groupBy("hack_license")

agg_exprs = []

# get the max value for each borough
for borough in unique_boroughs:
    agg_exprs.append(max_fun(col(borough)).alias(borough))

# Call .agg() with each column passed separately
agg_df = grouped_df.agg(
    agg_exprs[0],
    *agg_exprs[1:]
)

agg_df = agg_df.orderBy("hack_license")

agg_df.show(10)


25/05/15 16:50:29 WARN TaskSetManager: Stage 77 contains a task of very large size (8538 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+---------+------+--------+-----+-------------+
|        hack_license|Manhattan|Queens|Brooklyn|Bronx|Staten Island|
+--------------------+---------+------+--------+-----+-------------+
|0002555BBE359440D...|        0|     1|       0|    0|            0|
|000A4EBF1CEB9C6BD...|        0|     0|       0|    0|            0|
|000B8D660A329BBDB...|        2|     0|       1|    0|            0|
|000CCA239BFDC0ABE...|        2|     0|       0|    0|            0|
|00117D7CCD47D125E...|        2|     0|       0|    0|            0|
|0011B1575B9F5398B...|        2|     0|       0|    0|            0|
|0012703023AC1788D...|        2|     0|       0|    0|            0|
|001C8AAB90AEE49F3...|        2|     0|       0|    0|            0|
|0022A86EEFEE75C52...|        2|     0|       2|    0|            0|
|0023CF49F98E4F04A...|        2|     0|       0|    0|            0|
+--------------------+---------+------+--------+-----+-------------+
only showing top 10 rows



## Polars

### Exercise 5

Please work on the merged dataset of trips and fares and perform the following data cleaning tasks:

1. Remove trips with invalid locations (i.e. not in New York City);
3. Remove trips with invalid amounts:
    - Total amount must be greater than zero;
    - Total amount must correspond to the sum of all the other amounts.
5. Remove trips with invalid time:
    - Pick-up before drop-off;
    - Valid duration.

After each data cleaning task, report how many rows where removed. Finally report:
- Are there **duplicate trips**?
- How many trips remain after cleaning?

In [72]:
# load data and override schema for datetime columns
trip_data_df = pl.read_csv(
    "./datasets/trip_data.csv",
    infer_schema_length=250,
    has_header=True,
    schema_overrides={"pickup_datetime": pl.Datetime, "dropoff_datetime": pl.Datetime},
)
trip_fare_df = pl.read_csv(
    "./datasets/trip_fare.csv",
    infer_schema_length=250,
    has_header=True,
    schema_overrides={" pickup_datetime": pl.Datetime, " dropoff_datetime": pl.Datetime},
)

In [73]:
trip_fare_df.head(5)

medallion,hack_license,vendor_id,pickup_datetime,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
str,str,str,datetime[μs],str,f64,f64,f64,f64,f64,f64
"""89D227B655E5C82AECF13C3F540D4C…","""BA96DE419E711691B9445D6A6307C1…","""CMT""",2013-01-01 15:11:48,"""CSH""",6.5,0.0,0.5,0.0,0.0,7.0
"""0BD7C8F5BA12B88E0B67BED28BEA73…","""9FD8F69F0804BDB5549F40E9DA1BE4…","""CMT""",2013-01-06 00:18:35,"""CSH""",6.0,0.5,0.5,0.0,0.0,7.0
"""0BD7C8F5BA12B88E0B67BED28BEA73…","""9FD8F69F0804BDB5549F40E9DA1BE4…","""CMT""",2013-01-05 18:49:41,"""CSH""",5.5,1.0,0.5,0.0,0.0,7.0
"""DFD2202EE08F7A8DC9A57B02ACB81F…","""51EE87E3205C985EF8431D850C7863…","""CMT""",2013-01-07 23:54:15,"""CSH""",5.0,0.5,0.5,0.0,0.0,6.0
"""DFD2202EE08F7A8DC9A57B02ACB81F…","""51EE87E3205C985EF8431D850C7863…","""CMT""",2013-01-07 23:25:03,"""CSH""",9.5,0.5,0.5,0.0,0.0,10.5


In [74]:
print("Trip data columns: ", trip_data_df.columns)
print("Trip fare columns: ", trip_fare_df.columns)

Trip data columns:  ['medallion', 'hack_license', 'vendor_id', 'rate_code', 'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_time_in_secs', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
Trip fare columns:  ['medallion', ' hack_license', ' vendor_id', ' pickup_datetime', ' payment_type', ' fare_amount', ' surcharge', ' mta_tax', ' tip_amount', ' tolls_amount', ' total_amount']


In [75]:
# normalize column names by stripping whitespace
trip_data_df = trip_data_df.rename({name: name.strip() for name in trip_data_df.columns})
trip_fare_df = trip_fare_df.rename({name: name.strip() for name in trip_fare_df.columns})

In [76]:
# create joined dataframe as before
joined_df = trip_data_df.join(trip_fare_df, on=["medallion", "hack_license", "pickup_datetime"], how="inner")

In [77]:
joined_df.head(5)

medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,vendor_id_right,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
str,str,str,i64,str,datetime[μs],datetime[μs],i64,i64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64
"""89D227B655E5C82AECF13C3F540D4C…","""BA96DE419E711691B9445D6A6307C1…","""CMT""",1,"""N""",2013-01-01 15:11:48,2013-01-01 15:18:10,4,382,1.0,-73.978165,40.757977,-73.989838,40.751171,"""CMT""","""CSH""",6.5,0.0,0.5,0.0,0.0,7.0
"""0BD7C8F5BA12B88E0B67BED28BEA73…","""9FD8F69F0804BDB5549F40E9DA1BE4…","""CMT""",1,"""N""",2013-01-06 00:18:35,2013-01-06 00:22:54,1,259,1.5,-74.006683,40.731781,-73.994499,40.75066,"""CMT""","""CSH""",6.0,0.5,0.5,0.0,0.0,7.0
"""0BD7C8F5BA12B88E0B67BED28BEA73…","""9FD8F69F0804BDB5549F40E9DA1BE4…","""CMT""",1,"""N""",2013-01-05 18:49:41,2013-01-05 18:54:23,1,282,1.1,-74.004707,40.73777,-74.009834,40.726002,"""CMT""","""CSH""",5.5,1.0,0.5,0.0,0.0,7.0
"""DFD2202EE08F7A8DC9A57B02ACB81F…","""51EE87E3205C985EF8431D850C7863…","""CMT""",1,"""N""",2013-01-07 23:54:15,2013-01-07 23:58:20,2,244,0.7,-73.974602,40.759945,-73.984734,40.759388,"""CMT""","""CSH""",5.0,0.5,0.5,0.0,0.0,6.0
"""DFD2202EE08F7A8DC9A57B02ACB81F…","""51EE87E3205C985EF8431D850C7863…","""CMT""",1,"""N""",2013-01-07 23:25:03,2013-01-07 23:34:24,1,560,2.1,-73.97625,40.748528,-74.002586,40.747868,"""CMT""","""CSH""",9.5,0.5,0.5,0.0,0.0,10.5


In [78]:
# function to extract the min and max coordinates from a geojson file
def get_min_max_coordinates(
    geojson: dict[str, Any],
) -> tuple[float, float, float, float]:
    """
    Get the min/max coordinates from a geojson file.
    """
    # get all the coordinates from the geojson file
    coordinates: list[tuple[float, float]] = []
    for feature in geojson["features"]:
        coordinates.extend(feature["geometry"]["coordinates"][0])
        
    # get the min/max coordinates
    min_lon = min([point[0] for point in coordinates])
    max_lon = max([point[0] for point in coordinates])
    min_lat = min([point[1] for point in coordinates])
    max_lat = max([point[1] for point in coordinates])

    return min_lon, max_lon, min_lat, max_lat

nyc_boroughs_geojson_path = Path("./datasets/nyc-boroughs.geojson")

with nyc_boroughs_geojson_path.open("r") as f:
    json_data = f.read()
    nyc_boroughs_geo_data: dict[str, Any] = json.loads(json_data)
# get the min/max coordinates
min_lon, max_lon, min_lat, max_lat = get_min_max_coordinates(nyc_boroughs_geo_data)

print(f"Min lon: {min_lon}, Max lon: {max_lon}")
print(f"Min lat: {min_lat}, Max lat: {max_lat}")

Min lon: -74.25559136315215, Max lon: -73.70002020503293
Min lat: 40.4961339876118, Max lat: 40.91553277700519


In [79]:
# filter dataset for invalid coordinates
filtered_ny_df = joined_df.filter(
    pl.col("pickup_longitude") > min_lon,
    pl.col("pickup_longitude") < max_lon,
    pl.col("pickup_latitude") > min_lat,
    pl.col("pickup_latitude") < max_lat,
    pl.col("dropoff_longitude") > min_lon,
    pl.col("dropoff_longitude") < max_lon,
    pl.col("dropoff_latitude") > min_lat,
    pl.col("dropoff_latitude") < max_lat,
)
print("Data after filtering invalid coordinates: ", filtered_ny_df.shape[0])

Data after filtering invalid coordinates:  14478922


In [80]:
# filter dataframe when amounts are wrong
filtered_ny_df = filtered_ny_df.filter(
        pl.col("total_amount") > 0,
        pl.col("total_amount")
        == (
            pl.col("fare_amount")
            + pl.col("surcharge")
            + pl.col("mta_tax")
            + pl.col("tip_amount")
            + pl.col("tolls_amount")
        ),
    )
print("Data after filtering invalid amount: ", filtered_ny_df.shape[0])

Data after filtering invalid amount:  14211370


In [81]:
# filter dataframe when times are wrong
filtered_ny_df = filtered_ny_df.filter(
        pl.col("pickup_datetime") < pl.col("dropoff_datetime"),
        (pl.col("dropoff_datetime") - pl.col("pickup_datetime")).dt.total_seconds() == pl.col("trip_time_in_secs"),
        pl.col("trip_distance") > 0,
        pl.col("trip_time_in_secs") < 24 * 60 * 60
)
print("Data after filtering invalid trip time: ", filtered_ny_df.shape[0])

Data after filtering invalid trip time:  10341916


In [82]:
print("Number of records before filtering: ", joined_df.shape[0])
print("Number of records after filtering: ", filtered_ny_df.shape[0])
print("Deleted records: ", joined_df.shape[0] - filtered_ny_df.shape[0])

Number of records before filtering:  14776615
Number of records after filtering:  10341916
Deleted records:  4434699


In [83]:
# Check for duplicates based on "medallion", "hack_license", "pickup_datetime" and "dropoff_datetime"
duplicates = filtered_ny_df.filter(filtered_ny_df.select(["hack_license", "pickup_datetime", "dropoff_datetime"]).is_duplicated())
print("Number of duplicates: ", duplicates.shape[0])

filtered_ny_df = filtered_ny_df.unique(subset=["hack_license", "pickup_datetime", "dropoff_datetime"])

Number of duplicates:  8


### Exercise 6

Compute the **total revenue** (total_amount) grouped by:
- Pick-up hour of the day (0–23)
- Passenger count (group >=6 into “6+”)

Create a heatmap where:
- X-axis = hour
- Y-axis = passenger count group
- Cell value = average revenue per trip

In [84]:
filtered_ny_df_new_columns = filtered_ny_df.with_columns([
    # Extract pickup hour
    pl.col("pickup_datetime").dt.hour().alias("pickup_hour"),

    # Group passenger count: >=6 as "6+"
    pl.when(pl.col("passenger_count") >= 6)
      .then(pl.lit("6+"))
      .otherwise(pl.col("passenger_count").cast(pl.Utf8))
      .alias("passenger_number_per_group"),
])

In [85]:
filtered_ny_df_new_columns.head(5)

medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,vendor_id_right,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,pickup_hour,passenger_number_per_group
str,str,str,i64,str,datetime[μs],datetime[μs],i64,i64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,i8,str
"""CEB808F4AF2CEF231746860EC473B2…","""33542DB21D420110EFA8C2560F322E…","""VTS""",1,null,2013-01-22 05:36:00,2013-01-22 05:49:00,1,780,4.3,-73.924454,40.769444,-73.955406,40.776981,"""VTS""","""CSH""",15.0,0.5,0.5,0.0,0.0,16.0,5,"""1"""
"""C9ABCE3E24388294A2626A7AE463DF…","""212E36A8919F461F95C21FDDD7D1C0…","""VTS""",1,null,2013-01-20 22:54:00,2013-01-20 22:59:00,1,300,1.21,-74.001747,40.735512,-74.007271,40.741711,"""VTS""","""CSH""",6.0,0.5,0.5,0.0,0.0,7.0,22,"""1"""
"""0293108C734F9B1C3CA58A43BA6535…","""47DE01B4C27EE4F86831656DA5BA13…","""VTS""",1,null,2013-01-17 00:55:00,2013-01-17 01:14:00,1,1140,6.29,-73.990105,40.746601,-73.939438,40.723873,"""VTS""","""CRD""",21.0,0.5,0.5,4.3,0.0,26.3,0,"""1"""
"""23E5F1FDC96168E865BF7B4427C016…","""FC39D3F05496640DB9A7BC6AC13BC3…","""VTS""",1,null,2013-01-19 11:03:00,2013-01-19 11:11:00,1,480,1.68,-73.984505,40.779758,-73.999489,40.759296,"""VTS""","""CSH""",8.0,0.0,0.5,0.0,0.0,8.5,11,"""1"""
"""04C5991FD21FBCA6FAE4AF69961D08…","""255243D75757CA69E2270A525DABEF…","""CMT""",1,"""N""",2013-01-13 16:27:35,2013-01-13 16:39:15,1,700,3.0,-73.999985,40.733227,-73.989471,40.767017,"""CMT""","""CRD""",11.0,0.0,0.5,2.3,0.0,13.8,16,"""1"""


In [86]:
# group by pickup hour and passenger number and calculate the average total amount
grouped_df = filtered_ny_df_new_columns.group_by(["pickup_hour", "passenger_number_per_group"]).agg(
    pl.col("total_amount").mean().alias("avg_total_amount")
).sort(["pickup_hour", "passenger_number_per_group"])

In [87]:
grouped_df.head(5)

pickup_hour,passenger_number_per_group,avg_total_amount
i8,str,f64
0,"""0""",78.75
0,"""1""",14.639476
0,"""2""",14.525148
0,"""3""",14.01229
0,"""4""",13.819334


In [88]:

pandas_df = grouped_df.to_pandas()

pandas_df["pickup_hour"] = pandas_df["pickup_hour"].astype(str)
pandas_df["passenger_number_per_group"] = pandas_df["passenger_number_per_group"].astype(str)


source = ColumnDataSource(pandas_df)

mapper = LinearColorMapper(
    palette=Reds[9], 
    low=pandas_df["avg_total_amount"].min(), 
    high=pandas_df["avg_total_amount"].max()
)

pickup_hours_sorted = sorted(pandas_df["pickup_hour"].unique(), key=lambda x: int(x))
passenger_groups_sorted = sorted(pandas_df["passenger_number_per_group"].unique(), key=lambda x: (x == "6+", int(x) if x.isdigit() else 100))

p = figure(
    title="Average Revenue by Hour and Passenger Group",
    x_range=pickup_hours_sorted,
    y_range=passenger_groups_sorted,
    tools="",
    tooltips=[("Hour", "@pickup_hour"), ("Passengers", "@passenger_number_per_group"), ("Avg $", "@avg_total_amount{0.00}")]
)

p.rect(
    x="pickup_hour",
    y="passenger_number_per_group",
    width=1,
    height=1,
    source=source,
    line_color=None,
    fill_color=transform("avg_total_amount", mapper)
)

color_bar = ColorBar(
    color_mapper=mapper,
    ticker=BasicTicker(),
    location=(0, 0)
)
p.add_layout(color_bar, 'right')

p.xaxis.axis_label = "Pickup Hour"
p.yaxis.axis_label = "Passenger Count Group"

show(p)

### Exercise 7

Define an "anomalous trip" as one that satisfies at least two of the following:
- Fare per mile is above the 95th percentile
- Tip amount > 100% of fare
- trip_time_in_secs is less than 60 seconds but distance is more than 1 mile

Create a dataframe of anomalous trips and:
- Report how many such trips exist
- Create a scatterplot to visualize the anomaly metrics
- Describe the visualization identifying groups and outliers

In [89]:
# filter df for trips with trip_distance > 0
anomalous_df = filtered_ny_df.filter(
    pl.col("trip_distance") > 0
)

# get fare per mile
anomalous_df = filtered_ny_df.with_columns(
    (pl.col("fare_amount") / pl.col("trip_distance")).alias("fare_per_mile")
)

# calculate the 95th percentile of fare_per_mile
fare_per_mile_95th = anomalous_df["fare_per_mile"].quantile(0.95)

# Create 3 new columns to check for anomalies
anomalous_df = anomalous_df.with_columns(
    (pl.col("fare_per_mile") > fare_per_mile_95th).alias("wrong_fare_per_mile"),

    (pl.col("tip_amount") > pl.col("fare_amount")).alias("wrong_tip_amount"),

    ((pl.col("trip_time_in_secs") < 60) & (pl.col("trip_distance") > 1)).alias("wrong_trip_time"),
)

In [90]:
anomalous_df.head(5)

medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,vendor_id_right,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,fare_per_mile,wrong_fare_per_mile,wrong_tip_amount,wrong_trip_time
str,str,str,i64,str,datetime[μs],datetime[μs],i64,i64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,bool,bool,bool
"""CEB808F4AF2CEF231746860EC473B2…","""33542DB21D420110EFA8C2560F322E…","""VTS""",1,null,2013-01-22 05:36:00,2013-01-22 05:49:00,1,780,4.3,-73.924454,40.769444,-73.955406,40.776981,"""VTS""","""CSH""",15.0,0.5,0.5,0.0,0.0,16.0,3.488372,false,false,false
"""C9ABCE3E24388294A2626A7AE463DF…","""212E36A8919F461F95C21FDDD7D1C0…","""VTS""",1,null,2013-01-20 22:54:00,2013-01-20 22:59:00,1,300,1.21,-74.001747,40.735512,-74.007271,40.741711,"""VTS""","""CSH""",6.0,0.5,0.5,0.0,0.0,7.0,4.958678,false,false,false
"""0293108C734F9B1C3CA58A43BA6535…","""47DE01B4C27EE4F86831656DA5BA13…","""VTS""",1,null,2013-01-17 00:55:00,2013-01-17 01:14:00,1,1140,6.29,-73.990105,40.746601,-73.939438,40.723873,"""VTS""","""CRD""",21.0,0.5,0.5,4.3,0.0,26.3,3.338633,false,false,false
"""23E5F1FDC96168E865BF7B4427C016…","""FC39D3F05496640DB9A7BC6AC13BC3…","""VTS""",1,null,2013-01-19 11:03:00,2013-01-19 11:11:00,1,480,1.68,-73.984505,40.779758,-73.999489,40.759296,"""VTS""","""CSH""",8.0,0.0,0.5,0.0,0.0,8.5,4.761905,false,false,false
"""04C5991FD21FBCA6FAE4AF69961D08…","""255243D75757CA69E2270A525DABEF…","""CMT""",1,"""N""",2013-01-13 16:27:35,2013-01-13 16:39:15,1,700,3.0,-73.999985,40.733227,-73.989471,40.767017,"""CMT""","""CRD""",11.0,0.0,0.5,2.3,0.0,13.8,3.666667,false,false,false


In [91]:
# count the number of anomalies for each record and filter for those with at least 2 anomalies
anomalous_df = anomalous_df.filter(
    (
       ( pl.col("wrong_fare_per_mile").cast(pl.Int8)) + 
        (pl.col("wrong_tip_amount").cast(pl.Int8)) + 
       ( pl.col("wrong_trip_time").cast(pl.Int8))
    ) >= 2
)
print("Number of anomalous records: ", anomalous_df.shape[0])

Number of anomalous records:  1182


In [92]:
# Convert to ColumnDataSource for better performance
source = ColumnDataSource(anomalous_df.to_pandas())

# Create hover tool with detailed information
hover = HoverTool(
    tooltips=[
        ("Distance", "@trip_distance miles"),
        ("Fare/mile", "@fare_per_mile{$0.2f}"),
        ("Tip", "@tip_amount{$0.2f}"),
        ("Time", "@trip_time_in_secs{0} secs"),
        ("Total Fare", "@total_amount{$0.2f}")
    ],
    formatters={
        '@trip_distance': 'printf',
        '@fare_per_mile': 'numeral',
        '@tip_amount': 'numeral',
        '@total_amount': 'numeral'
    }
)

# Fare per mile vs trip distance
p1 = figure(
    title="Fare Anomalies",
    x_axis_label="Trip Distance (miles)",
    y_axis_label="Fare per Mile ($)",
    height=400,
    width=400,
    tools="",
    
)
p1.scatter(
    source=source,
    x="trip_distance",
    y="fare_per_mile",
    size=8,
    alpha=0.6,
    line_alpha=1
)

# Tip amount vs fare per mile
p2 = figure(
    title="Tip Anomalies",
    x_axis_label="Fare per Mile ($)",
    y_axis_label="Tip Amount ($)",
    height=400,
    width=400,
    tools="",
)
p2.scatter(
    source=source,
    x="fare_per_mile",
    y="tip_amount",
    size=8,
    alpha=0.6,
    line_alpha=1
)

# Time vs distance
p3 = figure(
    title="Time-Distance Anomalies",
    x_axis_label="Trip Distance (miles)",
    y_axis_label="Trip Time (seconds)",
    height=400,
    width=400,
    tools="",
    
)
p3.scatter(
    source=source,
    x="trip_distance",
    y="trip_time_in_secs",
    size=8,
    alpha=0.6,
    line_alpha=1
)
# Add hover tool to each plot
p1.add_tools(hover)
p2.add_tools(hover)
p3.add_tools(hover)

# Display the plots
show(row(p1, p2, p3))

### Exercise 8
For each driver (hack_license), calculate the **total profit per hour worked**, where:
> profit = 0.7 * (fare_amount + tip_amount) when the trip starts between 7:01 AM and 7:00 PM\
> profit = 0.8 * (fare_amount + tip_amount) when the trip starts between 7:01PM and 7:00 AM

Estimate "hours worked" by summing trip_time_in_secs.

Plot a line chart showing the distribution of average profit per hour **for the top 10% drivers** in terms of total trips.

Which time of day offers **best earning efficiency**?

In [93]:
# add new columns for profit calculation based on time of day and create a new column for trip hours
driver_profit_df = filtered_ny_df_new_columns.with_columns(
    pl.when(
        (pl.col("pickup_datetime").dt.time() >= pl.time(7, 1)) & 
        (pl.col("pickup_datetime").dt.time() <= pl.time(19, 0))
    )
    .then(0.7 * (pl.col("fare_amount") + pl.col("tip_amount")))
    .otherwise(0.8 * (pl.col("fare_amount") + pl.col("tip_amount")))
    .alias("total_profit"),
    (pl.col("trip_time_in_secs") / 3600).alias("trip_hours")
)

In [94]:
# group based on hack_license and pickup hour and calculate the total profit, total hours worked, and total trips, then calculate profit per hour and profit per trip
driver_profit_df = driver_profit_df.group_by("hack_license", "pickup_hour").agg(
    pl.col("total_profit").sum().alias("total_profit"),
    pl.col("trip_hours").sum().alias("total_hours_worked"),
    pl.col("trip_hours").count().alias("total_trips"),
).with_columns(
    (pl.col("total_profit") / pl.col("total_hours_worked")).round(2).alias("profit_per_hour"),
    (pl.col("total_profit") / pl.col("total_trips")).round(2).alias("profit_per_trip")
).sort("profit_per_hour", descending=True)

In [95]:
driver_profit_df.head(5)

hack_license,pickup_hour,total_profit,total_hours_worked,total_trips,profit_per_hour,profit_per_trip
str,i8,f64,f64,u32,f64,f64
"""003C68DFE1EBE120556D011948C788…",4,30.0,0.000278,1,108000.0,30.0
"""CF74B361ABBD77E73448BC536C6D6A…",2,16.4,0.000556,1,29520.0,16.4
"""5EE1CD0A797F45CA403AD5D4AB224E…",17,12.334,0.000556,1,22201.2,12.33
"""C6C26153997715375783FB29E9D100…",5,48.8,0.0025,1,19520.0,48.8
"""5EE1CD0A797F45CA403AD5D4AB224E…",15,32.2,0.001944,3,16560.0,10.73


In [96]:
# get the top 10% of drivers based on total trips
top_10_pct_threshold = driver_profit_df["total_trips"].quantile(0.9)
top_drivers = driver_profit_df.filter(pl.col("total_trips") >= top_10_pct_threshold)

In [97]:
# calculate the best hours for the top drivers
best_hours = top_drivers.group_by("pickup_hour").agg(
    (pl.col("total_profit").sum() / pl.col("total_hours_worked").sum()).round(2).alias("avg_profit_per_hour")
).sort("pickup_hour")

best_hours

pickup_hour,avg_profit_per_hour
i8,f64
0,58.52
1,60.25
2,61.39
3,63.43
4,66.96
…,…
19,51.28
20,54.94
21,56.61


In [98]:
best_hours_pd = best_hours.to_pandas()

# Create ColumnDataSource
source = ColumnDataSource(best_hours_pd)

# Create the plot
p = figure(
    title="Average Profit per Hour for Top 10% Drivers",
    x_axis_label='Hour of Day',
    y_axis_label='Average Profit per Hour ($)',
    x_range=(0, 23),
    width=800,
    height=400,
    tools=""
)

# Add hover tool
hover = HoverTool(
    tooltips=[
        ("Hour", "@pickup_hour:00"),
        ("Avg Profit/Hour", "$@avg_profit_per_hour{0.00}")
    ],
    mode='vline'
)
p.add_tools(hover)

# Add line plot
p.line(
    x='pickup_hour',
    y='avg_profit_per_hour',
    source=source,
    line_width=1,
    legend_label='Hourly Profit'
)

# Customize axes
p.xaxis.ticker = list(range(24))
p.xgrid.ticker = list(range(24))
p.yaxis.minor_tick_line_color = None

show(p)


In [99]:
# print the best hour for the top drivers
best_hour_row = best_hours.sort('avg_profit_per_hour', descending=True).row(0, named=True)
best_hour = f"{best_hour_row['pickup_hour']}:00"
print(f"The most profitable hour is at {best_hour} with ${best_hour_row['avg_profit_per_hour']} average profit per hour")

The most profitable hour is at 5:00 with $74.46 average profit per hour
